In [1]:
#Import functions and load data
import os
os.chdir("../src")
import tensorflow as tf
import numpy as np
from dataloader import qm9_parse, qm9_fetch
import dmol
import torch
import torch.nn as nn
qm9_records = qm9_fetch()
data = qm9_parse(qm9_records)


2023-11-26 15:59:43.075273: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-26 15:59:44.534928: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-26 15:59:44.534980: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-26 15:59:47.603433: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

Found existing record file, delete if you want to re-fetch


2023-11-26 16:00:00.166139: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-26 16:00:00.169037: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-26 16:00:00.169120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Katrines): /proc/driver/nvidia/version does not exist
2023-11-26 16:00:00.173009: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Take samples for test, validation and training
shuffled_data = data.shuffle(7000, reshuffle_each_iteration=False)
test_set = shuffled_data.take(1000)
valid_set = shuffled_data.skip(1000).take(1000)
train_set = shuffled_data.skip(2000).take(5000)

# for d in test_set:
#     print(d)
#     break


In [3]:
import torch
import torch.nn as nn

def convert_record(d, atom_types=100, embedding_dim=128):
    # break up record
    (e, x), y = d

    # Convert to PyTorch tensors
    e = torch.tensor(e.numpy())
    x = torch.tensor(x.numpy())
    r = x[:, :3]

    # Assuming atom indices start from 1
    e = e - 1
    e = torch.clamp(e, 0, atom_types - 1)  # Ensure indices are within valid range

    # Embedding
    embedding_layer = nn.Embedding(num_embeddings=atom_types, embedding_dim=embedding_dim)
    s = embedding_layer(e)

    return (s, r), y.numpy()[13]  # Select attribute at index 13


#
def x2e(x, cutoff_distance=5.0):
    """convert xyz coordinates to pairwise distance with a cutoff distance"""
   # Calculate pairwise distances
   # this calculates the norm
    r2 = torch.sqrt(((x - x[:, None, :])**2).sum(dim=-1))

    # Create a mask for distances less than cutoff_distance
    mask = (r2>0) & (r2 < cutoff_distance)

    # Use the mask to set values in the tensor
    r_ij = torch.where(mask, r2, torch.zeros_like(r2))

    # Generate edge index matrix
    #edge_index = torch.nonzero(mask, as_tuple=False)

    #edge_mask = (r2 > 0) & (r2 < cutoff_distance)
    edge_indices = mask.nonzero(as_tuple=True)
    edge_index = torch.stack(edge_indices)
    #edge_index = edge_index.resize_(2,len(mask))

    return r_ij, edge_index


In [4]:
# Assuming test_set is a list of data points
for d in test_set:
    (s, r_ij), y_raw = convert_record(d)

print(y_raw)
nodes = e
edges = x2e(x)

for d in test_set:
    (e, x), y = convert_record(d)
    r2, edge_index, = x2e(x)
    print("Edge Index:", edge_index)
    break  # To print only the first molecule


-320.75858
Edge Index: tensor([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,
          4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,
          6,  6,  6,  6,  6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,
          8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,
          9,  9,  9,  9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11,
         11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12,
         12, 12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
         13, 13],
        [ 1,  2,  3,  4,  7,  8,  9, 10, 11, 12, 13,  0,  2,  3,  4,  5,  6,  7,
          8,  9, 10, 11, 12, 13,  0,  1,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13,
          0,  1,  2,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  0,  1,

### Normalising the targets
Is working

In [6]:
#Normalize y values first and transform after prediction
ys = [convert_record(d)[1] for d in train_set]
train_ym = np.mean(ys)
train_ys = np.std(ys)
def transform_label(y):
    return (y - train_ym) / train_ys
def transform_prediction(y):
    return y * train_ys + train_ym


### Painn model

In [44]:
#PAINN model

from typing import Callable, Dict, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F

# import schnetpack.properties as properties
# import schnetpack.nn as snn

__all__ = ["phi", "RBF","F_cut","w", "u", "v","S", "PaiNN"]


# v_norm = r_ij/torch.sqrt(torch.sum(r_ij**2))

# v_j = torch.zeros(128)


class phi(nn.Module):
     def __init__(self,input_dim, s):
        super().__init__()
        self.input_dim=128
        self.s = s
        # self.input_dim=input_dim
        activation_fn = nn.SiLU
        self.net = nn.Sequential(
            nn.Linear(self.input_dim, 128),
            activation_fn(),
            nn.Linear(128, 384))

    # def forward(self,s):
    #     return self.net(s)

class RBF(nn.Module):
    def __init__(self, r_ij,r_cut):
        super().__init__()
        self.r_ij = r_ij
        self.r_cut = r_cut
        self.n_values = torch.arange(1, 21, dtype=torch.float32)
    def forward(self,r_ij, r_cut):
        r_RBF = torch.sin((self.n_values*torch.pi()/self.r_cut)*r_ij)/r_ij
        return r_RBF

class F_cut(nn.Module):
    def __init__(self, r_ij,r_cut):
        super().__init__()
        self.r_ij = r_ij
        self.r_cut = r_cut
    def forward(self, r_ij,r_cut):
        f_c=0.5*torch.cos(torch.pi()*r_ij/self.r_cut)+1
        return f_c

class w(nn.Module):
    def __init__(self,r_ij,r_cut):
        super().__init__()
        self.r_ij = r_ij
        self.r_cut = r_cut

        self.RBF=RBF(r_ij, 5.0) # mangler vi at importere functionen her??
        self.F_cut=F_cut(r_ij,5.0)
        self.net = nn.Linear(20,384)

    def forward(self,r_ij):
        New_RBF = self.RBF(r_ij, 5.0)
        New_F_cut=self.F_cut(r_ij, 5.0)
        Total = New_RBF*New_F_cut
        Output=self.net(Total)
        return Output


class MessageBlock(nn.Module):

    def __init__(self, v_j, s, r_ij):
        super().__init__()
        # if first pass, # v_j = torch.zeros(128) TODO
        self.phi = phi(128, s)
        self.w = w(r_ij=None, r_cut=5.0)  # Initialize w with r_ij=None

    def forward(self, s, r_ij, v_j, v_norm):
        output_phi = self.phi(s)
        output_w = self.w(r_ij)
        output_conv = output_phi * output_w
        output_split = torch.split(output_conv, 3, dim=1)  # Split along the second dimension

        # TODO: replace these with edge indexes from x2e
        # atom i will be updated as a function of it's j neighbors (atom j)
        v_j = torch.zeros(128)
        output_v=output_conv[0]*v_j
        output_r=output_conv[2]*(r_ij/r_ij) #TODO undersøg denne

        delta_s_im=torch.sum(output_conv[1],dim=1)
        delta_v_im = torch.sum(output_v+output_r,dim=1)

        return delta_s_im, delta_v_im


In [45]:
#tester data
# epochs=1

# for d in train_set:
#     (s, r), y_raw = convert_record(d)
#     y = transform_label(y_raw)
#     r_ij, edge_index = x2e(r, 5.0)
# v_j = torch.zeros(128)


In [50]:
# tester messageblock




MessageBlock(v_j, s, r_ij)

# delta_s_im, delta_v_im


MessageBlock(
  (phi): phi(
    (net): Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): SiLU()
      (2): Linear(in_features=128, out_features=384, bias=True)
    )
  )
  (w): w(
    (RBF): RBF()
    (F_cut): F_cut()
    (net): Linear(in_features=20, out_features=384, bias=True)
  )
)

In [ ]:
#Update block

class u(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Linear(128, 128)

    def forward(self, v_m):
        return self.net(v_m)


class v(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Linear(128, 128)

    def forward(self, v_m):
        return self.net(v_m)


class S(nn.Module):
    def __init__(self):
        super(S, self).__init__()
        activation_fn = nn.SiLU()
        self.net = nn.Sequential(
            nn.Linear(256, 128),
            activation_fn,
            nn.Linear(128, 384)
        )

    def forward(self, v_norm, s_m):
        stack = torch.stack((v_norm, s_m))
        output = self.net(stack)
        output = torch.split(output, 128)
        return output


class UpdateBlock(nn.Module):
    def __init__(self):
        super().__init__()
        self.s_m, self.v_m = MessageBlock() # Kat har added
        self.u = u()
        self.v = v()
        self.s = S()

    def forward(self, v_m, v_j, s_j, s_m):
        output_u = self.u(v_m)
        output_v = self.v(v_m)
        output_s = self.s(v_norm=s_m, s_m=s_j)

        V_dup = output_v.repeat(1, 2)  # Assuming v_m has shape (batch_size, 128)
        output_s1 = output_s[0] * output_u
        output_s2 = output_s[1] * V_dup
        output_s3 = output_s[2] + output_s2

        # TODO: replace these with edge indexes from x2e
        # atom i will be updated as a function of it's j neighbors (atom j)


        v_i = output_s1 + v_j
        s_i = output_s3 + s_j

        return v_i, s_i


### Final Painn model

Not working

In [ ]:
#Final PAINN model

# TODO: fix painn modellen
# KAT her

class PaiNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.message_block = MessageBlock()
        self.update_block = UpdateBlock()


        # You may need to define an initial input, for example, s_j, r_ij, v_j, and v_norm
        # Replace the following with your actual initialization logic
        self.initial_s_j = torch.zeros((batch_size, 128))
        self.initial_r_ij = torch.zeros((batch_size, 3))
        self.initial_v_j = torch.zeros((batch_size, 128))
        self.initial_v_norm = torch.zeros((batch_size, 3))

    def forward(self, num_iterations):
        # Initialize variables
        s_j = self.initial_s_j
        r_ij = self.initial_r_ij
        v_j = self.initial_v_j
        v_norm = self.initial_v_norm

        for _ in range(num_iterations):
            # Message block
            output1 = self.message_block.phi(s_j)
            output2 = self.message_block.w(r_ij)
            output = output1 * output2
            output_split = torch.split(output, 3, dim=1)

            # Update s_m
            s_m = torch.sum(output_split[1], dim=1, keepdim=True) + s_j

            # Update v_m
            output3 = output_split[2] * v_norm
            v_m = torch.sum(output3, dim=1, keepdim=True) + v_j

            # Update block
            output_u = self.update_block.u(v_m)
            output_v = self.update_block.v(v_m)
            output_s = self.update_block.s(v_norm=s_m, s_m=s_j)

            V_dup = output_v.repeat(1, 2)
            output_s1 = output_s[0] * output_u
            output_s2 = output_s[1] * V_dup
            output_s3 = output_s[2] + output_s2

            v_i = output_s1 + v_j
            s_i = output_s3 + s_j

            # Update variables for the next iteration
            s_j = s_i
            v_j = v_i

        # The final v_i and s_i after all iterations
        return v_i, s_i


In [ ]:
# Final PAINN model - Katrine


class PaiNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.message_block = MessageBlock()
        self.update_block = UpdateBlock()


class PaiNN(nn.Module):
    def __init__(self, phi_input_size, r_ij, r_cut, v_m_size, s_m_size):
        super().__init__()
        self.message_block = Message_block(phi_input_size, r_ij, r_cut)
        self.update_block = Update_block(v_m_size, s_m_size)


    def forward(self, input1, input2, v_j, s, v_norm):
        # Forward pass through the message block
        v_m, s_m = self.message_block(input1, input2, v_j, s, v_norm)

        # Forward pass through the update block
        v_u, s_u = self.update_block(v_m, v_m, s_m, v_m, s_m)

        # Return the updated values
        return v_u, s_u


### Training

In [ ]:
#How to iterate through the data
eta = 1e-3
val_loss = [0.0 for _ in range(epochs)]
epochs=3
for epoch in range(epochs):
    for d in train_set:
        (e, x), y_raw = convert_record(d)
        y = transform_label(y_raw)
        grad = loss_grad(e, x, y, w1, w2, w3, b)



        # TODO: Look at this, this is made by the chat
        # update regression weights
        w3 -= eta * grad[2]
        b -= eta * grad[3]
        # update GNN weights
        for i, w in [(0, w1), (1, w2)]:
            for j in range(len(w)):
                w[j] -= eta * grad[i][j] / 10
    # compute validation loss
    for v in valid_set:
        (e, x), y_raw = convert_record(v)
        y = transform_label(y_raw)
        # convert SE to RMSE
        val_loss[epoch] += loss(e, x, y, w1, w2, w3, b)
    val_loss[epoch] = jnp.sqrt(val_loss[epoch] / 1000)
    eta *= 0.9
plt.plot(baseline_val_loss, label="baseline")
plt.plot(val_loss, label="GNN")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Val Loss")
plt.show()


In [ ]:
ys = []
yhats = []
for v in valid_set:
    (e, x), y = convert_record(v)
    ys.append(y)
    yhat_raw = model(e, x, w1, w2, w3, b)
    yhats.append(transform_prediction(yhat_raw))


plt.plot(ys, ys, "-")
plt.plot(ys, yhats, ".")
plt.xlabel("Energy")
plt.ylabel("Predicted Energy")
plt.show()
